In [ ]:
import numpy as np
import xsimlab as xs

In [ ]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

In [ ]:
from phydra.models import GridXY_NP_Chemostat_model

In [ ]:
GridXY_NP_Chemostat_model

In [ ]:
GridXY_NP_Chemostat_model.visualize(show_inputs=True)

# supply time, parameters and output

this model allows any number of phytoplankton types (or size classes) to be initialized within each cell. For simplicity "P_num" is 1 here:

In [ ]:
in_ds = xs.create_setup(
    model=GridXY_NP_Chemostat_model,
    clocks={   
        'time': np.linspace(0,30,1000),
    },
    master_clock='time',
    input_vars={
        'Grid__x_dim':100,
        'Grid__y_dim':100,
        
        'MS':{
            'N_initval':0.01,
            'P_num':1,
            'P_initval':.1
        },
        
        'Mix__flowrate':1.,
        
        'GX__exchange_rate':1.5
        
    },
    output_vars={
        # state of components as output
        'N__state':'time',
        'P__state':'time',
        
        # fluxes stored for diagnostic purposes
        'Mix__N_input':'time'
    }
)

# Model Run

In [ ]:
from xsimlab.monitoring import ProgressBar

with ProgressBar(frontend='console'):
    with GridXY_NP_Chemostat_model:
        out_ds = in_ds.xsimlab.run()

In [ ]:
print(out_ds)

# Model output

below the 2d grid of our model is plotted, with the 'time' that can be controlled via the slider next to it

In [ ]:
import hvplot.xarray
import holoviews as hv

import matplotlib.pyplot as plt

## Nutrient

The nutrient has a small square influx in the middle of the grid, over time the first influx of nutrients is taken up by phytoplankton that consumes it, reaching a steady state quickly.

In [ ]:
N_out = out_ds.N__state.hvplot.image(
    x='x', y='y', clim=(0, 5),
    width=550, height=550,
    cmap=plt.cm.viridis, groupby='time'
)

N_out

## Phytoplankton

There is a small concentration of nutrient and phytoplankton initializedin each cell, that fuels a short growth across the grid. Additional nutrients flowing in at the middle are advected towards edges creating a gradient in phytoplankton biomass.

In [ ]:
P_out = out_ds.P__state.hvplot.image(
    x='x', y='y', clim=(0, 5),
    width=550, height=550,
    cmap=plt.cm.viridis, groupby='time'
)

P_out

## Nutrient input (as a model diagnostic)

In [ ]:
N_in = out_ds.Mix__N_input.hvplot.image(
    x='x', y='y', clim=(0, 5),
    width=550, height=550,
    cmap=plt.cm.viridis, groupby='time'
)

N_in